<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>

# 2.0 텍스트 분류자(Text Classifier) 구축하기

<img style="float: right;" src="images/nemo/nemo-app-stack.png" width=400>

이 노트북에서는 의학적 질병 abstract를 각각 암 질환, 신경 질환 및 장애, 그리고 기타의 세가지 분류 중 하나로 분류할 수 있는 애플리케이션을 구축해 봅니다. 여러분은 [NVIDIA NeMo](https://developer.nvidia.com/nvidia-nemo) (Neural Modules)을 활용하여 커맨드 라인에서 문제를 신속하게 설정하는 방법을 배워볼 예정입니다.  

**[2.1 NeMo 개요](#2.1-NeMo-개요)**<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.1.1 NeMo 모델](#2.1.1-NeMo-모델)<br>
**[2.2 커맨드 라인에서의 텍스트 분류](#2.2-커맨드-라인에서의-텍스트-분류)**<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.2.1 데이터 준비하기](#2.2.1-데이터-준비하기)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.2.2 환경 설정 파일(Configuration File)](#2.2.2-환경-설정-파일(Configuration-File))<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[2.2.2.1 OmegaConf 도구](#2.2.2.1-OmegaConf-도구)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.2.3 Hydra-Enabled Python 스크립트](#2.2.3-Hydra-Enabled-Python-스크립트)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.2.4 예제: Experiment 수행하기](#2.2.4-예제:-Experiment-수행하기)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.2.5 TensorBoard로 결과 시각화하기](#2.2.5-TensorBoard로-결과-시각화하기)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.2.6 예제: 언어 모델 변경하기](#2.2.6-예제:-언어-모델-변경하기)<br>
**[2.3 PyTorch Lightning 모델과 Trainer 워크플로우](#2.3-PyTorch-Lightning-모델과-Trainer-워크플로우)**<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.3.1 스크립트 주요 특징](#2.3.1-스크립트-주요-특징)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.3.2 처음(Scratch)부터 모델 트레이닝 하기](#2.3.2-처음(Scratch)부터-모델-트레이닝-하기)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.3.3 예제: 모델 질의하기](#2.3.3-예제:-모델-질의하기)<br>


---
# 2.1 NeMo 개요
NeMo는 대화형 AI 애플리케이션을 구축하기 위한 오플 소스 툴킷입니다. NeMo는 [Neural Modules](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/core/core.html#neural-modules)을 중심으로 구축되어, 유형화된 입력을 받아 유형화된 아웃풋으로 생성하는 뉴럴 네트워크의 개념 블록(conceptual blocks)으로 되어 있습니다. 이러한 모듈은 일반정으로 데이터 레이어, 인코더, 디코더, 언어 모델, 손실 함수, 또는 결합된 액티베이션 방법들을 나타냅니다. NeMo는 이러한 빌딩 블록들을 결합하고 재사용하기 쉽게 만들어주면서 Neural Type 시스템을 통해 의미론적 정확도 검사(a level of semantic correctness checking)를 제공합니다.

NeMo 딥 러닝 프레임워크는 뉴럴 네트워크 트레이닝을 위한 PyTorch 코드를 정리한 Pytorch wrapper인 [Pytorch Lightning](https://github.com/PyTorchLightning/pytorch-lightning)에 기반되어 있습니다. PyTorch Lightning은 쉽고 고성능의 멀티-GPU/멀티-노트 혼합 정밀 트레이닝(mixed precision training) 옵션을 제공합니다. 딥 뉴럴 네트워크 프로젝트 또는 **experiment(실험)**을 만들려면, 두 가지 주요 구성 요소가 필요합니다.:
1. [LightningModule(라이트닝모듈)](https://pytorch-lightning.readthedocs.io/en/stable/common/lightning_module.html)
2. [Trainer(트레이너)](https://pytorch-lightning.readthedocs.io/en/stable/common/trainer.html)

_LightningModule(라이트닝모듈)_ 은 트레이닝, 유효성 검사, 테스트를 위한 연산, 옵티마이저, 루프 문으로 PyTorch 코드로 구성하는데 활용합니다. 이 추상화 기법은 딥 러닝 실험을 이해하고 재생산하기 더 쉽게 만들어 줍니다. 

그리고 _Trainer(트레이너)_ 는 LightningModule(라이트닝모듈)을 가지고 올 수 있으며 딥 러닝 트레이닝을 위해 필요한 모든 것을 자동화할 수 있습니다. 

## 2.1.1 NeMo 모델

[NeMo 모델](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/core/core.html#)은 트레이닝과 재현성을 위한 모든 지원 인프라를 갖춘 LightningModules(라이트닝모듈) 입니다. 여기에는 딥 러닝 모델 아키텍처, 데이터 사전 처리, 옵티마이저, 체크포인트와 실험 로깅 기능이 포함됩니다. NeMo 모델은 라이트닝모듈과 같이 PyTorch 모듈이며 더 넓은 PyTorch 생태계와 완벽하게 호환됩니다. 어떤 NeMo model도 모든 PyTorch 워크플로우에 연결할 수 있습니다.   

**모든 NeMo 모델의 예제 Configuration 파일과 트레이닝 스크립트는 다음 [NVIDIA NeMo GitHub Repo](https://github.com/NVIDIA/NeMo/tree/main/examples)에서 찾을 수 있습니다.**

이번 수업을 위해서, [NGC NeMo container](https://ngc.nvidia.com/catalog/containers/nvidia:nemo)에 기반한 실습 환경에 포함된 로컬 repo 복사본를 이용할 계획이며 NLP 모델에 집중합니다. 다음 셀을 실행하여 `examples/nlp` 디렉토리에 있는 NeMo 모델 트리를 확인합니다. 

In [12]:
!tree nemo/examples/nlp -L 2

nemo/examples/nlp
├── dialogue_state_tracking
│   ├── conf
│   └── sgd_qa.py
├── entity_linking
│   ├── build_index.py
│   ├── conf
│   ├── data
│   ├── query_index.py
│   └── self_alignment_pretraining.py
├── glue_benchmark
│   ├── glue_benchmark.py
│   └── glue_benchmark_config.yaml
├── information_retrieval
│   ├── bert_dpr.py
│   ├── bert_joint_ir.py
│   ├── conf
│   ├── construct_random_negatives.py
│   └── get_msmarco.sh
├── intent_slot_classification
│   ├── conf
│   ├── data
│   └── intent_slot_classification.py
├── language_modeling
│   ├── bert_pretraining.py
│   ├── conf
│   ├── convert_weights_to_nemo1.0.py
│   ├── get_wkt2.sh
│   └── transformer_lm.py
├── machine_translation
│   ├── conf
│   ├── create_tarred_monolingual_dataset.py
│   ├── create_tarred_parallel_dataset.py
│   ├── enc_dec_nmt.py
│   ├── nmt_transformer_infer.py
│   ├── nmt_webapp
│   ├── preprocess_dataset.py
│   └── translate_ddp.py
├── question_answering
│   ├── conf
│   ├── get_squad.py
│   └── question

클래식한 NLP 태스크를 다루는 여러 모델이 있습니다. 이번 노트북에서는 [텍스트 분류](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/nlp/text_classification.html)에 대해 초점을 맞출 예정이며, 다음에 있을 명명된 엔티티 인식(NER) 노트북에서는 [토큰 분류](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/nlp/token_classification.html) 을 중심으로 실습을 진행할 예정입니다. 

각각의 NeMo 모델 타입은 환경 구성 파일에 대한 `conf` 폴더와 하나 이상의 Python 트레이닝 스크립트 파일을 포함하고 있습니다. 

텍스트 분류에 대한 자세한 내용을 보려면 다음 셀을 실행하십시오:

In [13]:
TC_DIR = "/dli/task/nemo/examples/nlp/text_classification"
!tree $TC_DIR

/dli/task/nemo/examples/nlp/text_classification
├── conf
│   └── text_classification_config.yaml
├── data
│   └── import_datasets.py
├── model_parallel_text_classification_evaluation.py
└── text_classification_with_bert.py

2 directories, 4 files


환경 파일 `text_classification_config.yaml`에서는 파일 위치, 사전 훈련된 모델, 하이퍼 파라미터와 같은 모델, 트레이닝, 실험 관리를 위한 세부 정보를 지정합니다. 

Python 스크립트 `text_classification_with_bert.py`는 환경 파일에서 정의된 텍스트 분류 실험을 진행하는데 필요한 모든 것을 캡슐화하고 있습니다. Configuration(환경 설정) 관리를 위해 페이스북의 [Hydra](https://hydra.cc/) 도구를 사용하여 커맨드 라인 옵션을 사용하여 구성 값을 재정의할 수 있도록 하여 스크립트와 함께 전체 실험을 진행할수 있도록 합니다!

실험을 신속하게 구축하는 비결은 기본 구성 파일에 포함된 내용과 여러분의 프로젝트를 위해 변경해야하는 내용을 정확히 이해하는 것입니다. 

---
# 2.2 커맨드 라인에서의 텍스트 분류
저희가 이번 실습에서 답변하고자 하는 질문은 다음과 같습니다.: 

**주어진 의학적 질병에 대한 Abstract는 암에 대한 것인가요? 아니면 신경 장애에 관련된 것인가요? 아니면 그 외의 것인가요?**

이 것은 3개의 등급 텍스트 분류 문제 입니다. 우리는 NeMo [텍스트 분류 모델](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/nlp/text_classification.html) 을 3가지 클래스(레이블) 로 활용할 예정입니다. - "cancer - 암" (0), "neurological disorders - 신경 장애" (1), and "other -기타" (2) 

## 2.2.1 데이터 준비하기
여러분은 이미 [NCBI-질병 코퍼스](https://www.ncbi.nlm.nih.gov/CBBresearch/Dogan/DISEASE/)와 [데이터 살펴보기](010_ExploreData.ipynb) 노트북으로부터 텍스트 분류 데이터 세트를 살펴보았습니다. 탭으로 구분된 abstracts와 레이블  헤더 행으로 되어 있는 텍스트 분류 파일을 기억해보세요.

In [14]:
TC3_DATA_DIR = '/dli/task/data/NCBI_tc-3'
!ls $TC3_DATA_DIR/*.tsv

/dli/task/data/NCBI_tc-3/dev.tsv
/dli/task/data/NCBI_tc-3/dev_nemo_format.tsv
/dli/task/data/NCBI_tc-3/test.tsv
/dli/task/data/NCBI_tc-3/test_nemo_format.tsv
/dli/task/data/NCBI_tc-3/train.tsv
/dli/task/data/NCBI_tc-3/train_nemo_format.tsv


In [15]:
# Take a look at the tab separated data
print("*****\ntrain.tsv sample\n*****")
!head -n 3 $TC3_DATA_DIR/train.tsv
print("\n\n*****\ndev.tsv sample\n*****")
!head -n 3 $TC3_DATA_DIR/dev.tsv
print("\n\n*****\ntest.tsv sample\n*****")
!head -n 3 $TC3_DATA_DIR/test.tsv

*****
train.tsv sample
*****
sentence	label
Identification of APC2, a homologue of the adenomatous polyposis coli tumour suppressor . The adenomatous polyposis coli ( APC ) tumour-suppressor protein controls the Wnt signalling pathway by forming a complex with glycogen synthase kinase 3beta ( GSK-3beta ) , axin / conductin and betacatenin . Complex formation induces the rapid degradation of betacatenin . In colon carcinoma cells , loss of APC leads to the accumulation of betacatenin in the nucleus , where it binds to and activates the Tcf-4 transcription factor ( reviewed in [ 1 ] [ 2 ] ) . Here , we report the identification and genomic structure of APC homologues . Mammalian APC2 , which closely resembles APC in overall domain structure , was functionally analyzed and shown to contain two SAMP domains , both of which are required for binding to conductin . Like APC , APC2 regulates the formation of active betacatenin-Tcf complexes , as demonstrated using transient transcriptional act


데이터의 몇 가지 기능을 참고하세요:
* 사전 처리된 데이터는 [documentation](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/nlp/text_classification.html)에 지정된 형식과 같이 이미 아래처럼 되어있습니다. 

   ```
   [WORD][SPACE][WORD][SPACE][WORD][TAB][LABEL]
   ``` 
  
* 헤더 행에는 "문장 레이블(sentence label)"이 있으며 제거되어야 합니다. 
* 텍스트가 상당히 길기 때문에 `max_seq_length` 값을 트레이닝 시 고려해야 합니다.

헤더 행을 제거하는 것부터 시작해 보세요. 이 작업을 수행하는 방법은 여러 가지가 있지만 간단한 변경 작업이므로 우리는 bash 스트림 에디터(`sed`) 명령어를 활용할 수 있습니다.

In [16]:
!sed 1d $TC3_DATA_DIR/train.tsv > $TC3_DATA_DIR/train_nemo_format.tsv
!sed 1d $TC3_DATA_DIR/dev.tsv > $TC3_DATA_DIR/dev_nemo_format.tsv
!sed 1d $TC3_DATA_DIR/test.tsv > $TC3_DATA_DIR/test_nemo_format.tsv

In [17]:
# Take a look at the tab separated data
# "1" is "positive" and "0" is "negative"
print("*****\ntrain_nemo_format.tsv sample\n*****")
!head -n 3 $TC3_DATA_DIR/train_nemo_format.tsv
print("\n\n*****\ndev_nemo_format.tsv sample\n*****")
!head -n 3 $TC3_DATA_DIR/dev_nemo_format.tsv
print("\n\n*****\ntest_nemo_format.tsv sample\n*****")
!head -n 3 $TC3_DATA_DIR/test_nemo_format.tsv

*****
train_nemo_format.tsv sample
*****
Identification of APC2, a homologue of the adenomatous polyposis coli tumour suppressor . The adenomatous polyposis coli ( APC ) tumour-suppressor protein controls the Wnt signalling pathway by forming a complex with glycogen synthase kinase 3beta ( GSK-3beta ) , axin / conductin and betacatenin . Complex formation induces the rapid degradation of betacatenin . In colon carcinoma cells , loss of APC leads to the accumulation of betacatenin in the nucleus , where it binds to and activates the Tcf-4 transcription factor ( reviewed in [ 1 ] [ 2 ] ) . Here , we report the identification and genomic structure of APC homologues . Mammalian APC2 , which closely resembles APC in overall domain structure , was functionally analyzed and shown to contain two SAMP domains , both of which are required for binding to conductin . Like APC , APC2 regulates the formation of active betacatenin-Tcf complexes , as demonstrated using transient transcriptional activa

In [18]:
TC3_DATA_DIR = '/dli/task/data/NCBI_tc-3'
!ls $TC3_DATA_DIR/*.tsv

/dli/task/data/NCBI_tc-3/dev.tsv
/dli/task/data/NCBI_tc-3/dev_nemo_format.tsv
/dli/task/data/NCBI_tc-3/test.tsv
/dli/task/data/NCBI_tc-3/test_nemo_format.tsv
/dli/task/data/NCBI_tc-3/train.tsv
/dli/task/data/NCBI_tc-3/train_nemo_format.tsv


## 2.2.2 환경 설정 파일(Configuration File)
`text_classification_config.yaml` config 파일을 확인하고 키와 기본 value값을 확인해 봅니다. 특히, 세가지 최상위 키인 `trainer`, `model`,`exp_manager`와 키의 계층 구조를 확인해 두세요.

```yaml
trainer:
  gpus:
  num_nodes:
  max_epochs:
  ...
  
model:
  nemo_path:
  tokenizer:  
  language_model:
  classifier_head:
  ...

exp_manager:
  ...
```

In [19]:
CONFIG_DIR = "/dli/task/nemo/examples/nlp/text_classification/conf"
CONFIG_FILE = "text_classification_config.yaml"
!cat $CONFIG_DIR/$CONFIG_FILE

# Copyright (c) 2020, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Config file for text classification with pre-trained BERT models

trainer:
  gpus: 1 # number of GPUs (0 for CPU), or list of the GPUs to use e.g. [0, 1]
  num_nodes: 1
  max_epochs: 100
  max_steps: null # precedence over max_epochs
  accumulate_grad_batches: 1 # accumulates grads every k batches
  gradient_clip_val: 0.0
  amp_level: O0 # O1/O2 for mixed precision
  precision: 32 # S

### 2.2.2.1 OmegaConf 도구
YAML config 파일은 파라미터 대부분의 기본 값을 제공하지만 텍스트 분류 실험을 진행하기 위해 지정되어야하는 몇 개의 파라미터가 있습니다.

각각의 YAML 섹션은 [omegaconf](https://omegaconf.readthedocs.io/en/2.1_branch/#) 패키지를 이용하면 보다 더 쉽게 확인할 수 있으며, "dot" 프로토콜을 활용하는 구성 키에 접근하고 조작할 수 있습니다.

Config 파일에서 `OmegaConf` 객체를 인스턴스화하는 것으로부터 시작합니다. 객체에 있는 키는 변경, 추가, 보기, 저장 등의 작업을 수행할 수 있습니다. 

예를 들어, `model`섹션만 보려고 하는 경우, 우리는 config 파일을 로드하여 `config.model` 섹션만 지정하여 프린트문을 통해 볼 수 있습니다. 

In [20]:
from omegaconf import OmegaConf

config = OmegaConf.load(CONFIG_DIR + "/" + CONFIG_FILE)
print(OmegaConf.to_yaml(config.model))

nemo_path: text_classification_model.nemo
tokenizer:
  tokenizer_name: ${model.language_model.pretrained_model_name}
  vocab_file: null
  tokenizer_model: null
  special_tokens: null
language_model:
  pretrained_model_name: bert-base-uncased
  lm_checkpoint: null
  config_file: null
  config: null
classifier_head:
  num_output_layers: 2
  fc_dropout: 0.1
class_labels:
  class_labels_file: null
dataset:
  num_classes: ???
  do_lower_case: false
  max_seq_length: 256
  class_balancing: null
  use_cache: false
train_ds:
  file_path: null
  batch_size: 64
  shuffle: true
  num_samples: -1
  num_workers: 3
  drop_last: false
  pin_memory: false
validation_ds:
  file_path: null
  batch_size: 64
  shuffle: false
  num_samples: -1
  num_workers: 3
  drop_last: false
  pin_memory: false
test_ds:
  file_path: null
  batch_size: 64
  shuffle: false
  num_samples: -1
  num_workers: 3
  drop_last: false
  pin_memory: false
optim:
  name: adam
  lr: 2.0e-05
  betas:
  - 0.9
  - 0.999
  weight_decay:

모델 인자(argument)에 대한 세부 사항은 다음 [문서](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/nlp/text_classification.html#model-arguments)에서 확인할 수 있습니다. 이 실습에서는 `dataset.num_classes` 값과 `train_ds.file_path`, `val_ds.file_path`, `test_ds.file_path` 에 있는 위치 값이 필요합니다.

메모리가 부족하지 않도록 `dataset.max_seq_length` 값을 128로 제한할 수 있습니다.  `infer_samples(추측 샘플)`이 영화 평점과 관련이 있는 것처럼 보일 수 있으므로 우리는 질병 도메인에서 의미 있는 문장으로 변경할 수 있습니다. 

나중에 변경해야할 여러가지 파라미터가 있지만 현재로서는 우리가 반드시 제공해야하는 모든 파라미터는 위와 같습니다. 

다음으로 `trainer(트레이너)` 하위 섹션을 살펴보겠습니다. 

In [21]:
print(OmegaConf.to_yaml(config.trainer))

gpus: 1
num_nodes: 1
max_epochs: 100
max_steps: null
accumulate_grad_batches: 1
gradient_clip_val: 0.0
amp_level: O0
precision: 32
accelerator: ddp
log_every_n_steps: 1
val_check_interval: 1.0
resume_from_checkpoint: null
num_sanity_val_steps: 0
checkpoint_callback: false
logger: false



현재 GPU가 하나 뿐이기 때문에, 이 설정은 괜찮긴 하지만 우선 `max_epochs`를 몇 개로 제한해야 할 수도 있습니다. `model`구성과 마찬가지로 몇 가지 다른 파라미터를 변경할 수도 있지만, 처음에는 기본값으로 가보도록 하겠습니다. 

마지막으로, `exp_manager`는 어떻습니까?

In [22]:
print(OmegaConf.to_yaml(config.exp_manager))

exp_dir: null
name: TextClassification
create_tensorboard_logger: true
create_checkpoint_callback: true



**이번** 섹션도 이대로 괜찮습니다. `exp_dir` 값이 `null`이라면  `nemo_experiments`라는 새 디렉터리에 실험 결과는 기본으로 배치될 예정입니다.

## 2.2.3 Hydra-Enabled Python 스크립트
정리를 하면, 우리가 변경 또는 재정의하고자 하는 파라미터는 다음과 같습니다.:

* `model.dataset.num_classes`: 3
* `model.dataset.max_seq_length`: 128
* `model.train_ds.file_path`: train_nemo_format.tsv
* `model.val_ds.file_path`: dev_nemo_format.tsv
* `model.test_ds.file_path`: test_nemo_format.tsv
* `model.infer_samples` : relevent sentences
* `trainer.max_epochs`: 3

텍스트 분류 트레이닝 스크립트를 이용해서 우리는 **하나의 명령문 만으로** 트레이닝, 추론, 테스트를 모두 진행할 수 있습니다!

스크립트는 Hydra를 이용하여 config 파일을 관리하므로 우리는 아래와 같이 커맨드 라인에서 직접 원하는 값을 다음과 같이 재정의할 수 있습니다.

In [12]:
%%time
# The training takes about 3 minutes to run

TC_DIR = "/dli/task/nemo/examples/nlp/text_classification"

# set the values we want to override
NUM_CLASSES = 3
MAX_SEQ_LENGTH = 128
PATH_TO_TRAIN_FILE = "/dli/task/data/NCBI_tc-3/train_nemo_format.tsv"
PATH_TO_VAL_FILE = "/dli/task/data/NCBI_tc-3/dev_nemo_format.tsv"
PATH_TO_TEST_FILE = "/dli/task/data/NCBI_tc-3/test_nemo_format.tsv"
# disease domain inference sample answers should be 0, 1, 2 
INFER_SAMPLES_0 = "In contrast no mutations were detected in the p53 gene suggesting that this tumour suppressor is not frequently altered in this leukaemia "
INFER_SAMPLES_1 = "The first predictive testing for Huntington disease  was based on analysis of linked polymorphic DNA markers to estimate the likelihood of inheriting the mutation for HD"
INFER_SAMPLES_2 = "Further studies suggested that low dilutions of C5D serum contain a factor or factors interfering at some step in the hemolytic assay of C5 rather than a true C5 inhibitor or inactivator"
MAX_EPOCHS = 3

# Run the training script, overriding the config values in the command line
!python $TC_DIR/text_classification_with_bert.py \
        model.dataset.num_classes=$NUM_CLASSES \
        model.dataset.max_seq_length=$MAX_SEQ_LENGTH \
        model.train_ds.file_path=$PATH_TO_TRAIN_FILE \
        model.validation_ds.file_path=$PATH_TO_VAL_FILE \
        model.test_ds.file_path=$PATH_TO_TEST_FILE \
        model.infer_samples=["$INFER_SAMPLES_0","$INFER_SAMPLES_1","$INFER_SAMPLES_2"] \
        trainer.max_epochs=$MAX_EPOCHS

[NeMo W 2024-03-04 02:28:59 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/omegaconf/basecontainer.py:225: UserWarning: cfg.pretty() is deprecated and will be removed in a future version.
    Use OmegaConf.to_yaml(cfg)
    
      warnings.warn(
    
[NeMo I 2024-03-04 02:28:59 text_classification_with_bert:110] 
    Config Params:
    trainer:
      gpus: 1
      num_nodes: 1
      max_epochs: 3
      max_steps: null
      accumulate_grad_batches: 1
      gradient_clip_val: 0.0
      amp_level: O0
      precision: 32
      accelerator: ddp
      log_every_n_steps: 1
      val_check_interval: 1.0
      resume_from_checkpoint: null
      num_sanity_val_steps: 0
      checkpoint_callback: false
      logger: false
    model:
      nemo_path: text_classification_model.nemo
      tokenizer:
        tokenizer_name: ${model.language_model.pretrained_model_name}
        vocab_file: null
        tokenizer_model: null
        special_tokens: null
      language_model:
        pretraine

각 트레이닝 실험이 시작할때, 커맨트 라인을 통해 추가되거나, 재정의된 파라미터를 포함하여 실험 사양(specification)의 로그가 프린트됩니다. 또한, 사용 가능한 GPU , 로그 저장 위치 및 모델에 대한 해당 입력과 함께 데이터세트로부터 일부 샘플과 같은 추가 정보도 함께 표시됩니다. 로그는 데이터세트에 있는 시퀀스 길이에 대한 일부 통계 값도 제공합니다.

각 epoch 후, 정밀도, 리콜, f1 점수를 포함한 검증 세트에 대한 메트릭 요약 테이블이 있습니다. f1 점수는 모든 false positive(위양성)와 false negative(위음성)을 모두 소고려하여 단순 정확도보다 더 유용한 것으로 간주됩니다.

트레이닝이 끝나면, NeMo는 `model.nemo_file_path`에서 지정한 경로에 마지막 체크포인트를 저장합니다.  이 값을 기본 값으로 두었기 때문에 `.nemo` 형식으로 워크스페이스에 기록될 예정입니다..

In [13]:
!ls *.nemo

text_classification_model.nemo


실험에서 얻은 결과가 그렇게 좋지 않으셨을 수 도 있습니다. 하지만 몇 가지 변경만으로 다른 실험을 시도하는 것은 매우 쉽습니다. 더 긴 트레이닝, 조정된 학습 속도, 트레이닝와 검증 데이터 세트의 배치 크기를 변경하면 결과를 개선할 수 있습니다. 

## 2.2.4 예제: Experiment 수행하기
동일한 텍스트 분류 문제를 사용하여 다른 유사한 실험을 진행해 봅니다. 이번에는 몇 가지 개선 사항이 제안되었습니다. :
  
* 혼합 정밀도를 `amp_level` 를 "O1" 로 16의 `precision(정밀도)` 로 설정하여 정확도에 있어 거의 또는 전혀 저하되지 않고 모델을 더 빠르게 실행할 수 있도록 합니다. 
* Epoch(에포크) 수를 약간 위로 조정하여 결과를 개선합니다. (더 큰 `max_epochs`)
* 학습 속도를 약간 높여 모델 가중치를 업데이트할 때 마다 예상 오류에 대해 보다 신속하게 대응할 수 있도록 합니다. 

아래 셀에 여러분을 위해 새로운 값이 제공되었습니다. 명령을 적절한 재정의와 함께 추가하고 셀을 실행합니다. 만약에 실습 진행에 어려움이 있으신 경우, [솔루션](solutions/ex2.2.4.ipynb)을 참고 합니다.

In [14]:
%%time
# The training takes about 3 minutes to run

TC_DIR = "/dli/task/nemo/examples/nlp/text_classification"

# set the values we want to override
NUM_CLASSES = 3
MAX_SEQ_LENGTH = 128
PATH_TO_TRAIN_FILE = "/dli/task/data/NCBI_tc-3/train_nemo_format.tsv"
PATH_TO_VAL_FILE = "/dli/task/data/NCBI_tc-3/dev_nemo_format.tsv"
PATH_TO_TEST_FILE = "/dli/task/data/NCBI_tc-3/test_nemo_format.tsv"
# disease domain inference sample answers should be 0, 1, 2 
INFER_SAMPLES_0 = "In contrast no mutations were detected in the p53 gene suggesting that this tumour suppressor is not frequently altered in this leukaemia "
INFER_SAMPLES_1 = "The first predictive testing for Huntington disease  was based on analysis of linked polymorphic DNA markers to estimate the likelihood of inheriting the mutation for HD"
INFER_SAMPLES_2 = "Further studies suggested that low dilutions of C5D serum contain a factor or factors interfering at some step in the hemolytic assay of C5 rather than a true C5 inhibitor or inactivator"
MAX_EPOCHS = 5
AMP_LEVEL = 'O1'
PRECISION = 16
LR = 5.0e-05

# Override the config values in the command line
!python $TC_DIR/text_classification_with_bert.py \
        model.dataset.num_classes=$NUM_CLASSES \
        model.dataset.max_seq_length=$MAX_SEQ_LENGTH \
        model.train_ds.file_path=$PATH_TO_TRAIN_FILE \
        model.validation_ds.file_path=$PATH_TO_VAL_FILE \
        model.test_ds.file_path=$PATH_TO_TEST_FILE \
        model.infer_samples=["$INFER_SAMPLES_0","$INFER_SAMPLES_1","$INFER_SAMPLES_2"] \
        trainer.max_epochs=$MAX_EPOCHS \
        trainer.amp_level=$AMP_LEVEL \
        trainer.precision=$PRECISION \
        model.optim.lr=$LR

[NeMo W 2024-03-04 02:33:58 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/omegaconf/basecontainer.py:225: UserWarning: cfg.pretty() is deprecated and will be removed in a future version.
    Use OmegaConf.to_yaml(cfg)
    
      warnings.warn(
    
[NeMo I 2024-03-04 02:33:58 text_classification_with_bert:110] 
    Config Params:
    trainer:
      gpus: 1
      num_nodes: 1
      max_epochs: 5
      max_steps: null
      accumulate_grad_batches: 1
      gradient_clip_val: 0.0
      amp_level: O1
      precision: 16
      accelerator: ddp
      log_every_n_steps: 1
      val_check_interval: 1.0
      resume_from_checkpoint: null
      num_sanity_val_steps: 0
      checkpoint_callback: false
      logger: false
    model:
      nemo_path: text_classification_model.nemo
      tokenizer:
        tokenizer_name: ${model.language_model.pretrained_model_name}
        vocab_file: null
        tokenizer_model: null
        special_tokens: null
      language_model:
        pretraine

이번 실험 결과가 이전 실험과 비교 했을 때 어떠셨나요? 출력 값에서 F1 점수와 추론 결과를 확인해 봅니다. 

## 2.2.5 TensorBoard로 결과 시각화하기
[experiment manager](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/core/core.html?highlight=tensorboard#experiment-manager) 는 텐서보드로 볼 수 있는 결과들을 저장합니다. <br>
인스턴스에 대한 [TensorBoard](/tensorboard/)를 열어 살펴보십시오.

실행한 모델들의 성능을 비고하려면 "Train Loss" 스케일러를 선택합니다. 여러분은 함께 실행한 모든 모델을 보실 수도 있고, 개별 모델을 선택하여 비교할 수 있습니다. 아래 예제는 첫 번째 주황색 실험 결과와 두 번째 파란색 예제 결과를 보여줍니다. 이를 통해 두 번째 실험에서 손실이 더 적었음을 알 수 있습니다. 

<img src="images/tensorboard_01.png" width=1000px>

## 2.2.6 예제: 언어 모델 변경하기
지금까지는 기본적인 `bert-base-uncased` 언어 모델을 사용해 왔지만, 이 모델은 여러분이 시도해볼 수 있는 모델 중 하나일 뿐입니다. 다음 셀을 실행하여 사용 가능한 언어 모델들을 확인해 보세요.

In [23]:
# complete list of supported BERT-like models
from nemo.collections import nlp as nemo_nlp
nemo_nlp.modules.get_pretrained_lm_models_list()

['megatron-bert-345m-uncased',
 'megatron-bert-345m-cased',
 'megatron-bert-uncased',
 'megatron-bert-cased',
 'biomegatron-bert-345m-uncased',
 'biomegatron-bert-345m-cased',
 'bert-base-uncased',
 'bert-large-uncased',
 'bert-base-cased',
 'bert-large-cased',
 'bert-base-multilingual-uncased',
 'bert-base-multilingual-cased',
 'bert-base-chinese',
 'bert-base-german-cased',
 'bert-large-uncased-whole-word-masking',
 'bert-large-cased-whole-word-masking',
 'bert-large-uncased-whole-word-masking-finetuned-squad',
 'bert-large-cased-whole-word-masking-finetuned-squad',
 'bert-base-cased-finetuned-mrpc',
 'bert-base-german-dbmdz-cased',
 'bert-base-german-dbmdz-uncased',
 'cl-tohoku/bert-base-japanese',
 'cl-tohoku/bert-base-japanese-whole-word-masking',
 'cl-tohoku/bert-base-japanese-char',
 'cl-tohoku/bert-base-japanese-char-whole-word-masking',
 'TurkuNLP/bert-base-finnish-cased-v1',
 'TurkuNLP/bert-base-finnish-uncased-v1',
 'wietsedv/bert-base-dutch-cased',
 'distilbert-base-uncased

이번 예제에서는 `megatron-bert-345m-cased`와 같은 새로운 언어 모델을 선택합니다.  

메모리를 지우기 위해 노트북 커널을 새로 시작하셔야 할 수도 있습니다. 대형 모델을 사용할 경우, GPU 메모리 공간을 절약하는 다른 방법은 `batch_size` 를 32, 16 또는 8로 줄이고 `max_seq_length`를 64로 줄이는 것입니다. 이 예제에서는 정답이 따로 없습니다. 오히려 이번 예제는 여러분이 여러 실험을 해볼 수 있는 기회입니다.일부 모델은 실행하는 데 몇 분이 소요될 수 있으므로 다음 노트북으로 먼저 이동한 후 나중에 시간이 나면 다시 여기로 돌아오셔도 좋습니다. 실습 진행에 어려움이 있는 경우, [예제 솔루션](solutions/ex2.2.6.ipynb) 을 살펴보십시오. 이 모델에 대한 손실값 및 f1 결과 값을 기록해놓거나 텐서보드를 통해 차이점을 시각화해보는 것을 잊지 마세요. 

In [16]:
# Restart the kernel
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [25]:
# TODO Try your own experiment with a different language model!

# The training takes about 10 minutes to run

TC_DIR = "/dli/task/nemo/examples/nlp/text_classification"

# set the values we want to override
NUM_CLASSES = 3
MAX_SEQ_LENGTH = 64
PATH_TO_TRAIN_FILE = "/dli/task/data/NCBI_tc-3/train_nemo_format.tsv"
PATH_TO_VAL_FILE = "/dli/task/data/NCBI_tc-3/dev_nemo_format.tsv"
PATH_TO_TEST_FILE = "/dli/task/data/NCBI_tc-3/test_nemo_format.tsv"
# disease domain inference sample answers should be 0, 1, 2 
INFER_SAMPLES_0 = "In contrast no mutations were detected in the p53 gene suggesting that this tumour suppressor is not frequently altered in this leukaemia "
INFER_SAMPLES_1 = "The first predictive testing for Huntington disease  was based on analysis of linked polymorphic DNA markers to estimate the likelihood of inheriting the mutation for HD"
INFER_SAMPLES_2 = "Further studies suggested that low dilutions of C5D serum contain a factor or factors interfering at some step in the hemolytic assay of C5 rather than a true C5 inhibitor or inactivator"
MAX_EPOCHS = 5
AMP_LEVEL = 'O1'
PRECISION = 16
LR = 5.0e-05
BATCH_SIZE = 32
PRETRAINED_MODEL_NAME = "megatron-bert-345m-cased"

!python $TC_DIR/text_classification_with_bert.py \
        model.dataset.num_classes=$NUM_CLASSES \
        model.dataset.max_seq_length=$MAX_SEQ_LENGTH \
        model.train_ds.file_path=$PATH_TO_TRAIN_FILE \
        model.validation_ds.file_path=$PATH_TO_VAL_FILE \
        model.test_ds.file_path=$PATH_TO_TEST_FILE \
        model.train_ds.batch_size=$BATCH_SIZE \
        model.validation_ds.batch_size=$BATCH_SIZE \
        model.test_ds.batch_size=$BATCH_SIZE \
        model.infer_samples=["$INFER_SAMPLES_0","$INFER_SAMPLES_1","$INFER_SAMPLES_2"] \
        trainer.max_epochs=$MAX_EPOCHS \
        trainer.amp_level=$AMP_LEVEL \
        trainer.precision=$PRECISION \
        model.optim.lr=$LR \
        model.language_model.pretrained_model_name=$PRETRAINED_MODEL_NAME

[NeMo W 2024-03-04 02:57:29 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/omegaconf/basecontainer.py:225: UserWarning: cfg.pretty() is deprecated and will be removed in a future version.
    Use OmegaConf.to_yaml(cfg)
    
      warnings.warn(
    
[NeMo I 2024-03-04 02:57:29 text_classification_with_bert:110] 
    Config Params:
    trainer:
      gpus: 1
      num_nodes: 1
      max_epochs: 5
      max_steps: null
      accumulate_grad_batches: 1
      gradient_clip_val: 0.0
      amp_level: O1
      precision: 16
      accelerator: ddp
      log_every_n_steps: 1
      val_check_interval: 1.0
      resume_from_checkpoint: null
      num_sanity_val_steps: 0
      checkpoint_callback: false
      logger: false
    model:
      nemo_path: text_classification_model.nemo
      tokenizer:
        tokenizer_name: ${model.language_model.pretrained_model_name}
        vocab_file: null
        tokenizer_model: null
        special_tokens: null
      language_model:
        pretraine

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



100% [..................................................] 669513499 / 669513499using world size: 1, data-parallel-size: 1, tensor-model-parallel size: 1, pipeline-model-parallel size: 1 
setting global batch size to 1
using torch.float32 for parameters ...
------------------------ arguments ------------------------
  adam_beta1 ...................................... 0.9
  adam_beta2 ...................................... 0.999
  adam_eps ........................................ 1e-08
  adlr_autoresume ................................. False
  adlr_autoresume_interval ........................ 1000
  apply_query_key_layer_scaling ................... True
  apply_residual_connection_post_layernorm ........ False
  attention_dropout ............................... 0.1
  attention_softmax_in_fp32 ....................... False
  bert_load ....................................... None
  bias_dropout_fusion ............................. False
  bias_gelu_fusion ................................ 

---
# 2.3 PyTorch Lightning 모델과 Trainer 워크플로우
NeMo 모델 스크립트는 가장 빠르게 시작하고 실행할 수 있는 방법입니다. 그러나 때로는 스크립트를 직접 작성하거나 프로젝트를 좀 더 맞춤화한 방식으로 수행하는 것이 더 좋습니다. 이 경우, 모델 트레이닝 스크립트 내에서 추상화된 (캡슐화되고 숨겨진) PyTorch 라이트닝 워크플로우를 단계별로 진행할 수 있습니다. 우리는 스크립트를 살펴 본 후에 스크립트 또는 Hydra 없이 처음부터 동일한 워크플로우로 진행해 보겠습니다. 

## 2.3.1 스크립트 주요 특징
[text_classification_with_bert.py](nemo/examples/nlp/text_classification/text_classification_with_bert.py) 스크립트를 열면 정확이 어떤 일들이 진행되고 있는 지를 확인할 수 있습니다. 

다음은 로깅 및 Initial comments가 제거된 축약된 버전입니다.:

```python
import pytorch_lightning as pl
from omegaconf import DictConfig

from nemo.collections.nlp.models.text_classification import TextClassificationModel
from nemo.collections.nlp.parts.nlp_overrides import NLPDDPPlugin
from nemo.core.config import hydra_runner
from nemo.utils.exp_manager import exp_manager


@hydra_runner(config_path="conf", config_name="text_classification_config")
def main(cfg: DictConfig) -> None:
    trainer = pl.Trainer(plugins=[NLPDDPPlugin()], **cfg.trainer)
    exp_manager(trainer, cfg.get("exp_manager", None))

    if not cfg.model.train_ds.file_path:
        raise ValueError("'train_ds.file_path' need to be set for the training!")

    model = TextClassificationModel(cfg.model, trainer=trainer)
    trainer.fit(model)

    if cfg.model.nemo_path:
        # '.nemo' file contains the last checkpoint and the params to initialize the model
        model.save_to(cfg.model.nemo_path)

    # We evaluate the trained model on the test set if test_ds is set in the config file
    if cfg.model.test_ds.file_path:
        trainer.test(model=model, ckpt_path=None, verbose=False)

    # perform inference on a list of queries.
    if "infer_samples" in cfg.model and cfg.model.infer_samples:       
        # max_seq_length=512 is the maximum length BERT supports.
        results = model.classifytext(queries=cfg.model.infer_samples, batch_size=16, max_seq_length=512)

if __name__ == '__main__':
    main()
```
Hydra 데코레이터 `@hydra_runner`는 환경 구성 파일을 연결하고 커맨드 라인의 재정의 메커니즘을 제공합니다.

환경 구성이 설정 되면, 주요 단계는 다음과 같습니다.:
1. 다음과 같이 트레이너를 인스턴스화 합니다. `trainer = pl.Trainer(plugins=[NLPDDPPlugin()], **cfg.trainer)`
1. 모델을 다음과 같이 인스턴스화 합니다. `model = TextClassificationModel(cfg.model, trainer=trainer)`
1. 모델을 다음과 함께 트레이닝 합니다. `trainer.fit(model)`

선택적인 추론과 평가를 위한 추가적인 단계는 다음과 같습니다.:
* 다음과 함께 평가를 진행합니다. `trainer.test(model=model, ckpt_path=None, verbose=False)`
* 다음과 함께 추론을 진행합니다. `results = model.classifytext(queries=cfg.model.infer_samples, batch_size=16, max_seq_length=512)`

## 2.3.2 처음(Scratch)부터 모델 트레이닝 하기
다음 셀을 실행하여 노트북 커널을 재시작하고 변수 및 GPU 메모리를 지웁니다.

In [26]:
# Restart the kernel
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

우선 `nemo_nlp` 컬렉션, experiment manager, PyTorch 라이트닝, 과 OmegaConf를 Import합니다.

In [2]:
from nemo.collections import nlp as nemo_nlp
from nemo.utils.exp_manager import exp_manager

import torch
import pytorch_lightning as pl
from omegaconf import OmegaConf

스크립트 또는 Hydra 없이 트레이닝 단계를 수동으로 실행할 경우, 인스턴스화 전에 올바른 구성이 설정되어 있어야 합니다.이 프로젝트의 구성 파일에 있어 변경하고자 하는 사항들이 이미 결정되었습니다. 이전에는 커맨드라인에 Hydra 재정의 기능을 사용했지만 이번에는 `OmegaConf`를 사용하여 변경하였습니다. `OmegaConf` 오브젝트인 `config` 를 파이썬에서 직접 변경하는 것을 제외하고 Syntax(구문)는 비슷해 보이며, 이 개체를 `trainer`, `exp_manager`, `model`로 전달할 것입니다..

기본 언어 모델은 `bert-base-uncased`입니다. 이 값을 재정의하려면, 아래 예시와 같이 셀에 추가 합니다:
```python
    PRETRAINED_MODEL_NAME = 'bert-base-cased'
    config.model.language_model.pretrained_model_name=PRETRAINED_MODEL_NAME
```

In [3]:
# Instantiate the OmegaConf object by loading the config file
TC_DIR = "/dli/task/nemo/examples/nlp/text_classification"
CONFIG_FILE = "text_classification_config.yaml"
config = OmegaConf.load(TC_DIR + "/conf/" + CONFIG_FILE)

# set the values we want to change
NUM_CLASSES = 3
MAX_SEQ_LENGTH = 128
PATH_TO_TRAIN_FILE = "/dli/task/data/NCBI_tc-3/train_nemo_format.tsv"
PATH_TO_VAL_FILE = "/dli/task/data/NCBI_tc-3/dev_nemo_format.tsv"
PATH_TO_TEST_FILE = "/dli/task/data/NCBI_tc-3/test_nemo_format.tsv"
# disease domain inference sample answers should be 0, 1, 2 
INFER_SAMPLES = ["Germline mutations in BRCA1 are responsible for most cases of inherited breast and ovarian cancer ",
        "The first predictive testing for Huntington disease  was based on analysis of linked polymorphic DNA markers to estimate the likelihood of inheriting the mutation for HD",
        "Further studies suggested that low dilutions of C5D serum contain a factor or factors interfering at some step in the hemolytic assay of C5 rather than a true C5 inhibitor or inactivator"
        ]
MAX_EPOCHS = 5
AMP_LEVEL = 'O1'
PRECISION = 16
LR = 5.0e-05

# set the config values using omegaconf
config.model.dataset.num_classes = NUM_CLASSES
config.model.dataset.max_seq_length = MAX_SEQ_LENGTH
config.model.train_ds.file_path = PATH_TO_TRAIN_FILE
config.model.validation_ds.file_path = PATH_TO_VAL_FILE
config.model.test_ds.file_path = PATH_TO_TEST_FILE
config.model.infer_samples = INFER_SAMPLES
config.trainer.max_epochs = MAX_EPOCHS
config.trainer.amp_level = AMP_LEVEL
config.trainer.precision = PRECISION
config.model.optim.lr = LR

이제 `config` 가 올바른 값으로 업데이트 되었으므로, 트레이너와 experiment manager를 인스턴스화 합니다. 

In [4]:
# Instantiate the trainer and experiment manager
trainer = pl.Trainer(**config.trainer)
exp_manager(trainer, config.exp_manager)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.


[NeMo I 2024-03-04 03:04:35 exp_manager:216] Experiments will be logged at /dli/task/nemo_experiments/TextClassification/2024-03-04_03-04-35
[NeMo I 2024-03-04 03:04:35 exp_manager:563] TensorboardLogger has been set up


PosixPath('/dli/task/nemo_experiments/TextClassification/2024-03-04_03-04-35')

In [5]:
# Instantiate the model 
model = nemo_nlp.models.TextClassificationModel(config.model, trainer=trainer)

Using bos_token, but it is not set yet.
Using eos_token, but it is not set yet.


[NeMo I 2024-03-04 03:04:41 text_classification_dataset:120] Read 683 examples from /dli/task/data/NCBI_tc-3/train_nemo_format.tsv.
[NeMo I 2024-03-04 03:04:41 text_classification_dataset:238] *** Example ***
[NeMo I 2024-03-04 03:04:41 text_classification_dataset:239] example 0: ['Familial', 'deficiency', 'of', 'the', 'seventh', 'component', 'of', 'complement', 'associated', 'with', 'recurrent', 'bacteremic', 'infections', 'due', 'to', 'Neisseria', '.', 'The', 'serum', 'of', 'a', '29-year', 'old', 'woman', 'with', 'a', 'recent', 'episode', 'of', 'disseminated', 'gonococcal', 'infection', 'and', 'a', 'history', 'of', 'meningococcal', 'meningitis', 'and', 'arthritis', 'as', 'a', 'child', 'was', 'found', 'to', 'lack', 'serum', 'hemolytic', 'complement', 'activity', '.', 'The', 'seventh', 'component', 'of', 'complement', '(', 'C7', ')', 'was', 'not', 'detected', 'by', 'functional', 'or', 'immunochemical', 'assays', ',', 'whereas', 'other', 'components', 'were', 'normal', 'by', 'hemolytic'

[NeMo W 2024-03-04 03:04:51 text_classification_dataset:250] Found 664 out of 683 sentences with more than 128 subtokens. Truncated long sentences from the end.


[NeMo I 2024-03-04 03:04:51 data_preprocessing:299] Some stats of the lengths of the sequences:
[NeMo I 2024-03-04 03:04:51 data_preprocessing:301] Min: 74 |                  Max: 129 |                  Mean: 128.36163982430455 |                  Median: 129.0
[NeMo I 2024-03-04 03:04:51 data_preprocessing:307] 75 percentile: 129.00
[NeMo I 2024-03-04 03:04:51 data_preprocessing:308] 99 percentile: 129.00
[NeMo I 2024-03-04 03:04:51 text_classification_dataset:120] Read 100 examples from /dli/task/data/NCBI_tc-3/dev_nemo_format.tsv.
[NeMo I 2024-03-04 03:04:51 text_classification_dataset:238] *** Example ***
[NeMo I 2024-03-04 03:04:51 text_classification_dataset:239] example 0: ['BRCA1', 'is', 'secreted', 'and', 'exhibits', 'properties', 'of', 'a', 'granin.', 'Germline', 'mutations', 'in', 'BRCA1', 'are', 'responsible', 'for', 'most', 'cases', 'of', 'inherited', 'breast', 'and', 'ovarian', 'cancer', '.', 'However', ',', 'the', 'function', 'of', 'the', 'BRCA1', 'protein', 'has', 'remai

[NeMo W 2024-03-04 03:04:52 text_classification_dataset:250] Found 99 out of 100 sentences with more than 128 subtokens. Truncated long sentences from the end.


[NeMo I 2024-03-04 03:04:52 data_preprocessing:299] Some stats of the lengths of the sequences:
[NeMo I 2024-03-04 03:04:52 data_preprocessing:301] Min: 120 |                  Max: 129 |                  Mean: 128.91 |                  Median: 129.0
[NeMo I 2024-03-04 03:04:52 data_preprocessing:307] 75 percentile: 129.00
[NeMo I 2024-03-04 03:04:52 data_preprocessing:308] 99 percentile: 129.00
[NeMo I 2024-03-04 03:04:52 text_classification_dataset:120] Read 10 examples from /dli/task/data/NCBI_tc-3/test_nemo_format.tsv.
[NeMo I 2024-03-04 03:04:52 text_classification_dataset:238] *** Example ***
[NeMo I 2024-03-04 03:04:52 text_classification_dataset:239] example 0: ['Clustering', 'of', 'missense', 'mutations', 'in', 'the', 'ataxia-telangiectasia', 'gene', 'in', 'a', 'sporadic', 'T-cell', 'leukaemia.', 'Ataxia-telangiectasia', '(', 'A-T', ')', 'is', 'a', 'recessive', 'multi-system', 'disorder', 'caused', 'by', 'mutations', 'in', 'the', 'ATM', 'gene', 'at', '11q22-q23', '(', 'ref', '.

[NeMo W 2024-03-04 03:04:52 text_classification_dataset:250] Found 10 out of 10 sentences with more than 128 subtokens. Truncated long sentences from the end.


[NeMo I 2024-03-04 03:04:52 data_preprocessing:299] Some stats of the lengths of the sequences:
[NeMo I 2024-03-04 03:04:52 data_preprocessing:301] Min: 129 |                  Max: 129 |                  Mean: 129.0 |                  Median: 129.0
[NeMo I 2024-03-04 03:04:52 data_preprocessing:307] 75 percentile: 129.00
[NeMo I 2024-03-04 03:04:52 data_preprocessing:308] 99 percentile: 129.00


[NeMo W 2024-03-04 03:04:52 modelPT:197] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact forit has already been registered.
[NeMo W 2024-03-04 03:04:52 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/nemo/core/classes/modelPT.py:243: UserWarning: update_node() is deprecated, use OmegaConf.update(). (Since 2.0)
      self.cfg.update_node(config_path, return_path)
    
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification m

In [6]:
%%time
# start model training and save result
# The training takes about 3 minutes to run
trainer.fit(model)
model.save_to(config.model.nemo_path)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2024-03-04 03:04:54 modelPT:748] Optimizer config = Adam (
    Parameter Group 0
        amsgrad: False
        betas: [0.9, 0.999]
        eps: 1e-08
        lr: 5e-05
        weight_decay: 0.01
    )
[NeMo I 2024-03-04 03:04:54 lr_scheduler:617] Scheduler "<nemo.core.optim.lr_scheduler.WarmupAnnealing object at 0x7fa12a19c8e0>" 
    will be used during training (effective maximum steps = 55) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: 0.1
    last_epoch: -1
    max_steps: 55
    )


initializing ddp: GLOBAL_RANK: 0, MEMBER: 1/1

  | Name                  | Type                 | Params
---------------------------------------------------------------
0 | loss                  | CrossEntropyLoss     | 0     
1 | bert_model            | BertEncoder          | 109 M 
2 | classifier            | SequenceClassifier   | 592 K 
3 | classification_report | ClassificationReport | 0     
---------------------------------------------------------------
110 M     Trainable params
0         Non-trainable params
110 M     Total params
440.301   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Epoch 3, global step 43: val_loss reached 0.58203 (best 0.58203), saving model to "/dli/task/nemo_experiments/TextClassification/2024-03-04_03-04-35/checkpoints/TextClassification--val_loss=0.58-epoch=3.ckpt" as top 3


Validating: 0it [00:00, ?it/s]

[NeMo I 2024-03-04 03:05:26 text_classification_model:165] val_report: 
    label                                                precision    recall       f1           support   
    label_id: 0                                             87.88      90.62      89.23         32
    label_id: 1                                             88.24      62.50      73.17         24
    label_id: 2                                             78.00      88.64      82.98         44
    -------------------
    micro avg                                               83.00      83.00      83.00        100
    macro avg                                               84.70      80.59      81.79        100
    weighted avg                                            83.62      83.00      82.63        100
    


Epoch 4, global step 54: val_loss reached 0.55868 (best 0.55868), saving model to "/dli/task/nemo_experiments/TextClassification/2024-03-04_03-04-35/checkpoints/TextClassification--val_loss=0.56-epoch=4.ckpt" as top 3
Saving latest checkpoint...
[NeMo W 2024-03-04 03:05:30 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/nemo/core/classes/modelPT.py:308: UserWarning: update_node() is deprecated, use OmegaConf.update(). (Since 2.0)
      conf.update_node(conf_path, item.path)
    


CPU times: user 1min 2s, sys: 16.7 s, total: 1min 19s
Wall time: 1min 26s


`config` 에서 업데이트한 테스트 세트의 파일 경로를 자동으로 사용하는 `trainer.test`으로 모델을 평가합니다. 

In [7]:
trainer.test(model=model, verbose=False)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

[NeMo I 2024-03-04 03:07:14 text_classification_model:165] test_report: 
    label                                                precision    recall       f1           support   
    label_id: 0                                             75.00      75.00      75.00          4
    label_id: 1                                            100.00     100.00     100.00          2
    label_id: 2                                             75.00      75.00      75.00          4
    -------------------
    micro avg                                               80.00      80.00      80.00         10
    macro avg                                               83.33      83.33      83.33         10
    weighted avg                                            80.00      80.00      80.00         10
    


[{'test_loss': 0.725864052772522,
  'test_precision': 79.99999237060547,
  'test_f1': 79.99999237060547,
  'test_recall': 79.99999237060547}]

마지막으로, `config`로 부터 추론 샘플을 활용하여 추론을 실행합니다. 우리는 `config.model.infer_samples` 키 오브젝트에서 직접 프린트를 통해 확인할 수 있습니다. 이 값은 문자열 목록으로 표시됩니다.

추론 또는 텍스트 분류를 실행하려면 `model.classifytext` 방법을 사용하십시오.추론된 라벨이 출력값으로 표시됩니다.

In [8]:
print(config.model.infer_samples)

['Germline mutations in BRCA1 are responsible for most cases of inherited breast and ovarian cancer ', 'The first predictive testing for Huntington disease  was based on analysis of linked polymorphic DNA markers to estimate the likelihood of inheriting the mutation for HD', 'Further studies suggested that low dilutions of C5D serum contain a factor or factors interfering at some step in the hemolytic assay of C5 rather than a true C5 inhibitor or inactivator']


In [9]:
model.classifytext(queries=config.model.infer_samples, batch_size=64, max_seq_length=128)

[0, 1, 2]

## 2.3.3 예제: 모델 질의하기
만약 추론을 위한 추가 쿼리를 지정하려고 하는 경우에는 어떻게 해야할까요? 방금 실습에 활용한 `model.classifytext` 메소드는 쿼리를 지정하지만 해당 쿼리는  구성 파일에 있지 _않아야_ 합니다. 우리는 간단히 우리가 원하는 질의에 대한 문자열 목록을 만들 수 있습니다. 

In [10]:
my_queries = [
    'Occasional missense mutations in ATM were also found in tumour DNA from patients with B-cell non-Hodgkins lymphomas.',
    'Myotonic dystrophy protein kinase is involved in the modulation of the Ca2+ homeostasis in skeletal muscle cells.',
    'Constitutional RB1-gene mutations in patients with isolated unilateral retinoblastoma.',
    'Hereditary deficiency of the fifth component of complement in man. I. Clinical, immunochemical, and family studies.'
]

`my_queries` 리스트에 있는 추론을 실행해 보세요. 실습에 어려움이 있는 경우, 다음[솔루션](solutions/ex2.3.3.ipynb)을 확인해 보세요.

In [11]:
# TODO Run inference over the my_queries list
model.classifytext(queries=my_queries, batch_size=16, max_seq_length=64)

[0, 1, 2, 2]

---
<h2 style="color:green;">축하합니다!</h2>

여러분은 3개의 클래스로 구성된 텍스트 분류기를 만들고 다음을 배웠습니다 :
* NeMo NLP 모델 구성 파일과 스크립트를 사용하여 빠르게 실험을 만드는 방법
* 환경 구성에 필요한 `model`, `trainer`, `exp_manager settings` 을 재정의 하는 방법
* 단일 커맨드 라인을 활용하여 텍스트 분류자를 트레이닝, 평가, 추론하는 방법
* PyTorch Lightning을 활용하여 텍스트 분류자를 트레이닝, 평가, 추론하는 방법

이제 다른 NLP 작업을 시도할 준비가 되었습니다.<br>

[3.0 명명된 엔티티 인식기 구축하기](030_NamedEntityRecognition.ipynb) 으로 이동합니다.

<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>